In [1]:
%load_ext autoreload
%autoreload 2
import torch
from PIL import Image
from pathlib import Path
import numpy as np

from robopose.config import LOCAL_DATA_DIR
from robopose.datasets.datasets_cfg import make_scene_dataset
from robopose.rendering.bullet_scene_renderer import BulletSceneRenderer
from robopose.visualization.singleview_articulated import make_singleview_prediction_plots
from robopose.visualization.bokeh_utils import save_image_figure

import os
#from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
from bokeh.plotting import gridplot
from bokeh.io import show, output_notebook; output_notebook()
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Setting OMP and MKL num threads to 1.
Project directory :/home/dario/Documents/RL_Git/robopose
pybullet build time: Dec  4 2023 09:43:03


Loading BokehJS ...

In [2]:
# Pick Robot
#renderer = BulletSceneRenderer('owi535')
#renderer = BulletSceneRenderer('panda')
# renderer = BulletSceneRenderer('baxter')
renderer = BulletSceneRenderer('iiwa7')

Loaded EGL 1.5 after reload.
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3070/PCIe/SSE2
GL_VERSION=4.6.0 NVIDIA 525.147.05
GL_SHADING_LANGUAGE_VERSION=4.60 NVIDIA
Version = 4.6.0 NVIDIA 525.147.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3070/PCIe/SSE2


EGL device choice: 0 of 3 (from EGL_VISIBLE_DEVICES)


In [3]:
# Pick inference result_id and dataset

# result_id = 'craves-lab-unknownq--1804'
# dataset = 'craves.lab.real.test'

result_id = 'dream-kuka-unknownq--1804'
dataset = 'dream.kuka.synt.photo.test'

#result_id = 'craves-youtube-unknownq-focal=1000--1804'
#dataset = 'craves.youtube'

print(LOCAL_DATA_DIR)

scene_ds = make_scene_dataset(dataset)

results = torch.load(LOCAL_DATA_DIR / 'results' / result_id / f'dataset={dataset}' / 'results.pth.tar')

/home/dario/Documents/RL_Git/robopose/local_data
[Memory]0.9s, 0.0min    : Loading build_frame_index...


In [10]:
# Generate visualization for a few images
type_joints = 'unknown'
dets = 'full_image_detections'


#pred_keys = [    f'{dets}/{type_joints}_joints/init',]

#pred_keys += [f'{dets}/{type_joints}_joints/iteration={K}' for K in (1, 10)]
#pred_keys += [f'{type_joints}_joints/iteration={K}' for K in (1, 10)]
pred_keys = [f'{dets}/{type_joints}_joints/iteration={K}' for K in (1,10) ]
#print(pred_keys)
#print(results['predictions']['full_image_detections/unknown_joints/iteration=10'].tensors['joints'])
#print(results['predictions'])

n_images = len(results['predictions'][f'{dets}/{type_joints}_joints/init'])
pred_ids = np.random.choice(np.arange(n_images), 5)
#10,12, 6,4
pred_ids=np.array([10,12, 6,4])
#print(f"Showing IDs: {pred_ids}")
fig_array = []
save_dir = Path(f'unknown_joints_dataset={dataset}')

fig_idx = 0
all_figures = []
def add_figure(fig):
    fig.title.text = f'fig={len(all_figures)}'
    all_figures.append(fig)
    
for pred_idx in tqdm(pred_ids):
    row = []
    for n, pred_key in enumerate(pred_keys):
        #print(f"N :{n}")
        #print(f"pred_key: {pred_key}")
        if pred_key in results['predictions']:
            
            
            pred = results['predictions'][pred_key][[pred_idx]]
            #print(f'Pred :{pred.infos.info()}')
            print(f"Predictions: {pred.tensors['poses']}")
            figures = make_singleview_prediction_plots(scene_ds, renderer, pred)
            if n == 0:
                fig = figures['input_im']
                add_figure(fig)
                row.append(fig)
            fig = figures['pred_overlay']
            add_figure(fig)
            row.append(fig)
        rgb_input = figures['rgb_input']
        rgb_overlay = figures['rgb_overlay']
    fig_array.append(row)
show(gridplot(fig_array, sizing_mode='scale_width'))

Setting OMP and MKL num threads to 1.
Project directory :/home/dario/Documents/RL_Git/robopose


Predictions: tensor([[[-0.0629, -0.9951, -0.0757, -0.0088],
         [-0.4337,  0.0956, -0.8960,  0.2134],
         [ 0.8989, -0.0236, -0.4376,  1.4446],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]])


ValueError: 

In [5]:
# Save selected figures
#show(all_figures[0])

save_ids = np.arange(len(all_figures))
from bokeh.io.export import export_png
from bokeh.plotting import figure, save
from selenium import webdriver

# Set up the Chrome WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode (no GUI)
driver = webdriver.Chrome(options=chrome_options)

for idx in save_ids:
    #save_image_figure(all_figures[idx], f'images/examples_{idx}.jpg')
    export_png(all_figures[idx], f'/home/dario/Pictures/examples_{idx}.png', webdriver=driver)
    #save(all_figures[idx], filename=f'/home/dario/Pictures/examples_{idx}.jpg', title='My Bokeh Plot')
  
# Close the WebDriver
driver.quit()  
    
 